How to use the LAG() value-based window function to extract particular column values from the previous row.
How the results from LAG() can be used to perform analysis such as calculating the rate of change between consecutive values.

In [1]:
%load_ext sql

In [3]:
#Testing connection to DB

import pymysql
import pandas as pd

# Create connection
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='Diagnostic_56',
    database='united_nations'
)

# Query
query = "SELECT * FROM united_nations.access_to_basic_services;"
df = pd.read_sql(query, conn)
df

C:\Users\User\AppData\Local\Temp\ipykernel_19148\4061088695.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,Region,Sub_region,Country_name,Time_period,Pct_managed_drinking_water_services,Pct_managed_sanitation_services,Est_population_in_millions,Est_gdp_in_billions,Land_area,Pct_unemployment
0,Central and Southern Asia,Central Asia,Kazakhstan,2015,94.67,98.00,17.542806,184.39,2699700.0,4.93
1,Central and Southern Asia,Central Asia,Kazakhstan,2016,94.67,98.00,17.794055,137.28,2699700.0,4.96
2,Central and Southern Asia,Central Asia,Kazakhstan,2017,95.00,98.00,18.037776,166.81,2699700.0,4.90
3,Central and Southern Asia,Central Asia,Kazakhstan,2018,95.00,98.00,18.276452,179.34,2699700.0,4.85
4,Central and Southern Asia,Central Asia,Kazakhstan,2019,95.00,98.00,18.513673,181.67,2699700.0,4.80
...,...,...,...,...,...,...,...,...,...,...
1043,Sub-Saharan Africa,Western Africa,Togo,2016,67.33,18.33,7.661354,6.03,54390.0,NaN
1044,Sub-Saharan Africa,Western Africa,Togo,2017,68.33,18.33,7.852795,6.40,54390.0,3.74
1045,Sub-Saharan Africa,Western Africa,Togo,2018,69.33,19.33,8.046679,7.11,54390.0,NaN
1046,Sub-Saharan Africa,Western Africa,Togo,2019,70.33,19.67,8.243094,7.22,54390.0,NaN


In [4]:
#Retrieve columns to be used

import pymysql
import pandas as pd

# Create connection
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='Diagnostic_56',
    database='united_nations'
)

# Query
query = "SELECT Country_name, Time_period, Pct_managed_drinking_water_services FROM united_nations.Access_to_Basic_Services;"
df = pd.read_sql(query, conn)
df

C:\Users\User\AppData\Local\Temp\ipykernel_19148\961835759.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,Country_name,Time_period,Pct_managed_drinking_water_services
0,Kazakhstan,2015,94.67
1,Kazakhstan,2016,94.67
2,Kazakhstan,2017,95.00
3,Kazakhstan,2018,95.00
4,Kazakhstan,2019,95.00
...,...,...,...
1043,Togo,2016,67.33
1044,Togo,2017,68.33
1045,Togo,2018,69.33
1046,Togo,2019,70.33


We apply the LAG() function as follows:

Firstly, this will partition our dataset by country, that is, the Country_name column, then order each partition by year, that is, the Time_period column, from the lowest to the highest.

Then, the LAG() function will be used to extract the previous year's percentage of managed drinking water within a country partition and based on the resulting order of rows from the ORDER BY clause. The results will then be stored in a new column Prev_year_pct_managed_drinking_water_services.

In [5]:


#1. Add a new column with the previous year's percentage of managed drinking water.

import pymysql
import pandas as pd

# Create connection
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='Diagnostic_56',
    database='united_nations'
)

# Query
query = "SELECT Country_name, Time_period, Pct_managed_drinking_water_services, LAG(Pct_managed_drinking_water_services) OVER( PARTITION BY Country_name ORDER BY Time_period ASC) AS Prev_year_pct_managed_drinking_water_services FROM united_nations.Access_to_Basic_Services;"
df = pd.read_sql(query, conn)
df

C:\Users\User\AppData\Local\Temp\ipykernel_19148\2787505721.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,Country_name,Time_period,Pct_managed_drinking_water_services,Prev_year_pct_managed_drinking_water_services
0,Afghanistan,2015,67.00,NaN
1,Afghanistan,2016,69.67,67.00
2,Afghanistan,2017,72.33,69.67
3,Afghanistan,2018,75.33,72.33
4,Afghanistan,2019,78.00,75.33
...,...,...,...,...
1043,Zimbabwe,2016,69.33,70.00
1044,Zimbabwe,2017,69.00,69.33
1045,Zimbabwe,2018,68.67,69.00
1046,Zimbabwe,2019,68.33,68.67


Let’s go further and determine the Annual Rate of Change between consecutive years. That is, the difference between Pct_managed_drinking_water_services and Prev_year_pct_managed_drinking_water_services.

Therefore, we subtract the previous percentage of managed drinking water from the current percentage of managed drinking water.

The query will work similarly to the previous one, except that we reuse the LAG() function to calculate the Annual Rate of Change.

In [8]:
#2. Determine the Annual Rate of Change between consecutive years.

import pymysql
import pandas as pd

# Create connection
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='Diagnostic_56',
    database='united_nations'
)

# Query
query = "SELECT Country_name, Time_period, Pct_managed_drinking_water_services, LAG(Pct_managed_drinking_water_services) OVER( PARTITION BY Country_name ORDER BY Time_period ASC) AS Prev_year_pct_managed_drinking_water_services, Pct_managed_drinking_water_services - LAG(Pct_managed_drinking_water_services) OVER( PARTITION BY Country_name ORDER BY Time_period ASC) AS ARC_pct_managed_drinking_water_services FROM united_nations.Access_to_Basic_Services LIMIT 80;;"
df = pd.read_sql(query, conn)
df

C:\Users\User\AppData\Local\Temp\ipykernel_19148\1976168638.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,Country_name,Time_period,Pct_managed_drinking_water_services,Prev_year_pct_managed_drinking_water_services,ARC_pct_managed_drinking_water_services
0,Afghanistan,2015,67.00,NaN,NaN
1,Afghanistan,2016,69.67,67.00,2.67
2,Afghanistan,2017,72.33,69.67,2.66
3,Afghanistan,2018,75.33,72.33,3.00
4,Afghanistan,2019,78.00,75.33,2.67
...,...,...,...,...,...
75,Barbados,2017,98.00,98.00,0.00
76,Barbados,2018,99.00,98.00,1.00
77,Barbados,2019,99.00,99.00,0.00
78,Barbados,2020,99.00,99.00,0.00


There is a new column containing the Annual Rate of Change values calculated based on the Pct_managed_drinking_water_services column values and the Prev_year_pct_managed_drinking_water_services column values.

For instance, we can see that in Afghanistan, in the year 2017, the percentage of managed drinking water services increased to 72.33 from 69.67 in the previous year, resulting in an Annual Rate of Change of 2.66.

In Barbados, the percentage of managed drinking water services in 2019 remained the same as the previous year, and therefore the Annual Rate of Change was 0